In [47]:
def fixed_parts_graphs(n0, n1):
    if n0==0:
        yield biflag(n1, 0)
        return
    if n1==0:
        yield biflag(n0, n0)
        return
    from sage.features.nauty import NautyExecutable
    import subprocess, select
    import shlex
    directg_path = NautyExecutable("genbg").absolute_filename()
    options="-l {} {}".format(n0, n1)
    sub = subprocess.Popen(
        shlex.quote(directg_path) + ' {0}'.format(options),
        shell=True,
        stdout=subprocess.PIPE,
        stdin=subprocess.PIPE,
        stderr=subprocess.PIPE,
        encoding='latin-1'
    )
    sub.stdin.close()
    if n0!=n1:
        for line in sub.stdout:
            g = Graph(line, format="graph6")
            yield biflag(n0+n1, n0, edges=tuple(g.edges(labels=False)))
    else:
        uniques = []
        for line in sub.stdout:
            g = Graph(line, format="graph6")
            can = tuple(g.canonical_label().edges(labels=None))
            if can not in uniques:
                uniques.append(can)
                yield biflag(n0+n1, n0, edges=tuple(g.edges(labels=False)))
    sub.wait()

def bipartite_graphs(n):
    for split in range(n//2 + 1):
        for xx in fixed_parts_graphs(split, n-split):
            yield xx

class biflag:
    def __init__(self, n, split, edges=None, tps=None):
        self.n = n
        self.edges = list(edges) if edges!=None else []
        self.tps = list(tps) if tps!=None else []
        self.split = split
        self.unique = None
        if split>n//2:
            self.edges = [(n-p0, n-p1) for p0, p1 in self.edges]
            self.tps = [n-ii for ii in self.tps]
            self.split = n-split

    def __str__(self):
        return "Biflag on {} split {} tps {} edges [{}]".format( \
            self.n, self.split, self.tps, " ".join(["{}{}".format(ee[0], ee[1]) for ee in self.edges]))

    __repr__ = __str__
    
    def subflag(self, n=None, tps=None):
        if tps==None:
            tps = self.tps
        if n==None:
            n = list(range(self.n))
        else:
            n = list(set(list(n)+list(tps)))
        if len(n)==self.n and tps==self.tps:
            return self
        n.sort()
        new_split = len([xx for xx in n if xx<split])
        new_edges = [(n.index(p0), n.index(p1)) for p0, p1 in self.edges if (p0 in n and p1 in n)]
        new_tps = [n.index(ii) for ii in tps]
        return biflag(len(n), new_split, edges=new_edges, tps=new_tps)
    
    def unique(self):
        if self.unique != None:
            return self.unique
        else:
            nsum = self.n
            V = list(nsum+1)
            E = self.edges + [(nsum, ii) for ii in range(self.split)]
            P = [[ii] for ii in self.tps] + [[jj for jj in range(nsum) if jj not in self.tps]] + [[nsum]]
            g = Graph([V, E], format="vertices_and_edges")
            can = g.canonical_label(partition=P)
            unique = (self.n, self.split, len(self.tps), tuple(can.edges(labels=None)))
            self.unique = unique
            return unique

    def __eq__(self, other):
        if self.n!=other.n or self.split!=other.split or len(self.edges)!=len(other.edges) or len(self.tps)!=len(other.tps):
            return False
        return self.unique()==other.unique()

def cont_matrix(ls1, ls2):
    

In [48]:
list(bipartite_graphs(4))

[Biflag on 4 split 0 tps [] edges [],
 Biflag on 4 split 1 tps [] edges [],
 Biflag on 4 split 1 tps [] edges [01],
 Biflag on 4 split 1 tps [] edges [01 02],
 Biflag on 4 split 1 tps [] edges [01 02 03],
 Biflag on 4 split 2 tps [] edges [],
 Biflag on 4 split 2 tps [] edges [03],
 Biflag on 4 split 2 tps [] edges [03 13],
 Biflag on 4 split 2 tps [] edges [02 13],
 Biflag on 4 split 2 tps [] edges [02 03 13],
 Biflag on 4 split 2 tps [] edges [02 03 12 13]]